In [1]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow import keras
from pathlib import Path
import tensorflow as tf
import yfinance as yf
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [2]:
# Download historical stock data for a specific ticker
ticker = 'AAPL'
stock_data = yf.download(ticker, start='2015-01-01', end='2020-01-01')

[*********************100%%**********************]  1 of 1 completed


In [3]:
display(stock_data)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.347172,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.661276,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.663494,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,23.995317,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,24.917267,237458000
...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,68.898697,48478800
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.265663,93121200
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.239006,146266000


In [4]:
features = stock_data[['Close']]
target = stock_data['Close']  # Predicting next time step temperature

In [5]:
scaler = MinMaxScaler()

In [6]:
features_scaled = scaler.fit_transform(features)

In [7]:
target_reshaped = target.values.reshape(-1, 1)

In [8]:
target_scaled = scaler.fit_transform(target_reshaped)

In [9]:
x = []
y = []
time_steps = 30

for i in range(time_steps, len(features_scaled)):
    x.append(features_scaled[i-time_steps:i])
    current_close_price = target_scaled[i]
    previous_close_price = target_scaled[i-1]
    if current_close_price >= previous_close_price:
        y.append(0)
    elif current_close_price == previous_close_price:
        y.append(1)
    elif current_close_price < previous_close_price:
        y.append(2)

X, y = np.array(x), np.array(y)

In [10]:
# Function to build and train a Long Short Term Memory Deep Learning memory
def BuildModel(X_train, y_train):
    # Define the model architecture
    model = Sequential()
    # Use Input layer to specify the shape
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    # Add LSTM layer
    model.add(LSTM(units=128, activation='relu'))
    # Output layer with 3 neurons (for 3 classes) and softmax activation
    model.add(Dense(3, activation='softmax'))  # 3 possible classes
    # Print model summary
    model.summary()
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # Fit the model with the training data
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.3)

    return model

In [11]:
X_train, X_test, y_train, y_test = X[:800], X[800:], y[:800], y[800:]

In [12]:
print(len(X_train)/len(X))
print(len(X_test)/len(X))

0.6514657980456026
0.3485342019543974


In [13]:
model = BuildModel(X_train, y_train)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 66,947 (261.51 KB)

 Trainable params: 66,947 (261.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 121ms/step - accuracy: 0.5375 - loss: 1.0730 - val_accuracy: 0.5250 - val_loss: 0.8335
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5214 - loss: 0.8008 - val_accuracy: 0.4750 - val_loss: 1.4008
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4853 - loss: 0.7058 - val_accuracy: 0.5000 - val_loss: 0.7777
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5453 - loss: 0.6939 - val_accuracy: 0.4750 - val_loss: 1.4583
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5029 - loss: 0.7083 - val_accuracy: 0.4750 - val_loss: 0.7592
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5580 - loss: 0.6998 - val_accuracy: 0.4750 - val_loss: 0.7803
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.4597 - loss: 0.7039 - val_accuracy: 0.4750 - val_loss: 0.7527
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5210 - loss: 0.7069 - val_accuracy: 0.5250 - 

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Loss: {test_loss}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4517 - loss: 0.7355
Test Accuracy: 0.4439
Test Loss: 0.7728655934333801


In [15]:
# Get predictions from the model (probabilities for each class)
predictions = model.predict(X_test)

# Convert the probabilities to class labels by selecting the class with the highest probability
predictions = np.argmax(predictions, axis=1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step


In [16]:
# Accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')

# Precision, Recall, F1 Score (calculated for each class)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, average='weighted')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
cm = confusion_matrix(y_test, predictions)
print(f"True Negatives: {cm[0][0]}")
print(f"True Positives: {cm[1][1]}")
print(f"False Negatives: {cm[1][0]}")
print(f"False Positives: {cm[0][1]}")

Accuracy: 0.4439
Precision: 0.1971
Recall: 0.4439
F1 Score: 0.2730
True Negatives: 0
True Positives: 190
False Negatives: 0
False Positives: 238


C:\Users\mghun\miniconda3\envs\sms\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
print(len(X_train)/len(X))
print(len(X_test)/len(X))

0.6995114006514658
0.3004885993485342


In [19]:
model = BuildModel(X_train, y_train)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 128)                 │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 66,947 (261.51 KB)

 Trainable params: 66,947 (261.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 119ms/step - accuracy: 0.4914 - loss: 1.0569 - val_accuracy: 0.4574 - val_loss: 0.8772
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4840 - loss: 0.8183 - val_accuracy: 0.4496 - val_loss: 0.7191
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5373 - loss: 0.7026 - val_accuracy: 0.4574 - val_loss: 0.7087
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.4948 - loss: 0.7078 - val_accuracy: 0.5426 - val_loss: 0.6933
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4931 - loss: 0.7015 - val_accuracy: 0.5426 - val_loss: 0.6969
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5141 - loss: 0.7024 - val_accuracy: 0.5426 - val_loss: 0.6909
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5202 - loss: 0.6940 - val_accuracy: 0.5426 - val_loss: 0.6915
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5229 - loss: 0.6957 - val_accuracy: 0.4574 - 

In [20]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Loss: {test_loss}')

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5137 - loss: 0.6934
Test Accuracy: 0.4878
Test Loss: 0.6941739916801453


In [21]:
# Get predictions from the model (probabilities for each class)
predictions = model.predict(X_test)

# Convert the probabilities to class labels by selecting the class with the highest probability
predictions = np.argmax(predictions, axis=1)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step


In [22]:
# Accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')

# Precision, Recall, F1 Score (calculated for each class)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, average='weighted')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix
cm = confusion_matrix(y_test, predictions)
print(f"True Negatives: {cm[0][0]}")
print(f"True Positives: {cm[1][1]}")
print(f"False Negatives: {cm[1][0]}")
print(f"False Positives: {cm[0][1]}")

Accuracy: 0.4878
Precision: 0.4824
Recall: 0.4878
F1 Score: 0.4832
True Negatives: 116
True Positives: 64
False Negatives: 106
False Positives: 83
